In [2]:
# %pip install segment-geospatial groundingdino-py leafmap localtileserver

In [1]:
from get_file_matches import get_als_tif_matches, get_als_dop_matches, get_tif_file_matches
import os

In [2]:
from samgeo import split_raster
from samgeo.text_sam import LangSAM
#import leafmap


In [3]:
def visualize_overlays(matches, max_images=3):
    """
    Visualize segmentation overlays using Leafmap for the first N images.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and corresponding TIF image paths as values.
    max_images (int): Maximum number of TIF images to visualize.
    """

    # Create a Leafmap interactive map
    m = leafmap.Map(center=[-22.1278, -51.4430], zoom=16, height="800px")

    # Add the Satellite Basemap
    m.add_basemap("SATELLITE")

    count = 0  # Counter to control the maximum number of visualizations
    for als_file, tif_file in matches.items():
        # Extract file identifier from the ALS file path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Check if the TIF file exists
        if not os.path.exists(tif_file):
            print(f"TIF file not found for {file_identifier}. Skipping visualization.")
            continue

        # Add the TIF file as a raster layer
        try:
            m.add_raster(tif_file, layer_name=f"Segmentation Mask ({file_identifier})", opacity=0.3)
        except Exception as e:
            print(f"Error adding TIF file to map for {file_identifier}: {e}")
            continue

        print(f"Added segmentation mask for {file_identifier} to the map.")

        # Break after visualizing the maximum number of images
        count += 1
        if count >= max_images:
            break

    print(f"Displaying map with {count} segmentation overlays.")
    return m  # Return the map to visualize the images


In [4]:
def create_tiles_from_matches(matches, tile_out_dir="tiles", tile_size=(1000, 1000), overlap=0):
    """
    Create tiles for TIF images based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and TIF image paths as values.
    tile_out_dir (str): Path to the output directory for tiles (default: "tiles").
    tile_size (tuple): Size of tiles for raster splitting (default: (1000, 1000)).
    overlap (int): Overlap between tiles (default: 0).
    """
    for als_file, dop_image in matches.items():
        # Extract file identifier from the ALS file path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Output directory for tiles of this TIF image
        tile_dir = os.path.join(tile_out_dir, file_identifier)

        # Ensure dop_image is a string, not a list
        if isinstance(dop_image, list):
            dop_image = dop_image[0]

        # Create tiles from the TIF image
        try:
            split_raster(filename=dop_image, out_dir=tile_dir, tile_size=tile_size, overlap=overlap)
            print(f"Tiles created for {file_identifier}. Saved in {tile_dir}.")
        except Exception as e:
            print(f"Error creating tiles for {file_identifier}: {e}")


In [5]:
def apply_sam_to_tiles_from_matches(matches, tile_base_dir="tiles", mask_out_dir="masks", text_prompt="tree", box_threshold=0.24, text_threshold=0.24):
    """
    Apply LangSAM to tiles based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and DOP image paths as values.
    tile_base_dir (str): Path to the base directory containing tiles.
    mask_out_dir (str): Path to the output directory for masks (default: "masks").
    text_prompt (str): Text prompt for LangSAM (default: "tree").
    box_threshold (float): Box threshold for LangSAM (default: 0.24).
    text_threshold (float): Text threshold for LangSAM (default: 0.24).
    """
    # Instantiate LangSAM once
    sam = LangSAM()

    for als_file, dop_image in matches.items():
        # Extract file identifier from the DOP image path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Path to the tiles directory for this DOP image
        tile_path = os.path.join(tile_base_dir, file_identifier)

        # Output directory for masks of this tile group
        mask_dir = os.path.join(mask_out_dir, file_identifier)

        # Apply LangSAM to the tiles
        sam.predict_batch(
            images=tile_path,
            out_dir=mask_dir,
            text_prompt=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            dtype="float32",
            mask_multiplier=1,
            merge=True,
            verbose=True,
        )

        print(f"Masks created for tiles in {tile_path}. Saved in {mask_dir}.")

In [3]:
# Root directory containing all subfolders
root_dir = "data/Tschernitz"
als_subfolder="als"
tif_subfolder="output\dem_chm_slope_aspect_bands"
# Match ALS files with TIF files that contain "merged"
als_tif_matches = get_als_tif_matches(root_dir,als_subfolder=als_subfolder, tif_subfolder=tif_subfolder, contains='stacked_7bands')

📂 Extracted identifier '33470-5714' from folder: als_33470-5714
✅ ALS File: data\Tschernitz\als\als_33470-5714\als_33470-5714.laz -> Matched TIF: data\Tschernitz\output\dem_chm_slope_aspect_bands\als_33470-5714\als_33470-5714_stacked_7bands.tif

📂 Extracted identifier '33470-5715' from folder: als_33470-5715
✅ ALS File: data\Tschernitz\als\als_33470-5715\als_33470-5715.laz -> Matched TIF: data\Tschernitz\output\dem_chm_slope_aspect_bands\als_33470-5715\als_33470-5715_stacked_7bands.tif

📂 Extracted identifier '33470-5716' from folder: als_33470-5716
✅ ALS File: data\Tschernitz\als\als_33470-5716\als_33470-5716.laz -> Matched TIF: data\Tschernitz\output\dem_chm_slope_aspect_bands\als_33470-5716\als_33470-5716_stacked_7bands.tif

📂 Extracted identifier '33471-5713' from folder: als_33471-5713
✅ ALS File: data\Tschernitz\als\als_33471-5713\als_33471-5713.laz -> Matched TIF: data\Tschernitz\output\dem_chm_slope_aspect_bands\als_33471-5713\als_33471-5713_stacked_7bands.tif

📂 Extracted iden

In [4]:
als_folder = "data/Tschernitz/als"
dop_folder = "data/Tschernitz/dop"

als_dop_matches = get_als_dop_matches(als_folder, dop_folder)

In [8]:
# Define output directory for tiles
tile_output_directory = "data/Tschernitz/tiles/dop_2500"

# Create tiles using the matches dictionary
create_tiles_from_matches(als_dop_matches, tile_out_dir=tile_output_directory,  tile_size=(2500, 2500), overlap=0)

Tiles created for als_33470-5714. Saved in data/Tschernitz/tiles/dop_2500\als_33470-5714.
Tiles created for als_33470-5715. Saved in data/Tschernitz/tiles/dop_2500\als_33470-5715.
Tiles created for als_33470-5716. Saved in data/Tschernitz/tiles/dop_2500\als_33470-5716.
Tiles created for als_33471-5713. Saved in data/Tschernitz/tiles/dop_2500\als_33471-5713.
Tiles created for als_33471-5714. Saved in data/Tschernitz/tiles/dop_2500\als_33471-5714.
Tiles created for als_33471-5715. Saved in data/Tschernitz/tiles/dop_2500\als_33471-5715.
Tiles created for als_33471-5716. Saved in data/Tschernitz/tiles/dop_2500\als_33471-5716.
Tiles created for als_33472-5713. Saved in data/Tschernitz/tiles/dop_2500\als_33472-5713.
Tiles created for als_33472-5714. Saved in data/Tschernitz/tiles/dop_2500\als_33472-5714.
Tiles created for als_33472-5715. Saved in data/Tschernitz/tiles/dop_2500\als_33472-5715.
Tiles created for als_33472-5716. Saved in data/Tschernitz/tiles/dop_2500\als_33472-5716.
Tiles crea

In [9]:
text_prompt="forest"

# Define output directory for masks
mask_output_directory = f"data/Tschernitz/ground_truth_masks/forest_masks_dop_2500"

In [17]:
# Get the first element from the dictionary
first_element = list(als_dop_matches.items())[0]

# Convert the first element to another dictionary
first_match_dict = {first_element[0]: first_element[1]}

In [10]:
# Apply SAM using the matches dictionary
apply_sam_to_tiles_from_matches(als_dop_matches, tile_base_dir=tile_output_directory, 
                                mask_out_dir=mask_output_directory,
                                box_threshold=0.24, text_threshold=0.24)

final text_encoder_type: bert-base-uncased
Processing image 1 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5714\tile_0_0.tif...
Processing image 2 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5714\tile_0_1.tif...
Processing image 3 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5714\tile_1_0.tif...
Processing image 4 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5714\tile_1_1.tif...
Saved the merged prediction to data/Tschernitz/ground_truth_masks/forest_masks_dop_2500\als_33470-5714\merged.tif.
Masks created for tiles in data/Tschernitz/tiles/dop_2500\als_33470-5714. Saved in data/Tschernitz/ground_truth_masks/forest_masks_dop_2500\als_33470-5714.
Processing image 1 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5715\tile_0_0.tif...
Processing image 2 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5715\tile_0_1.tif...
Processing image 3 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5715\tile_1_0.tif...
Processing image 4 of 4: data/Tschernitz/tiles/dop_2500\als_33470-5715\tile_1

In [5]:
# Root directory containing all subfolders
root_dir = "data/Tschernitz"
als_subfolder="als"
masks_subfolder="ground_truth_masks/forest_masks_dop_2500"
# Match ALS files with TIF files that contain "merged"
als_gt_matches = get_als_tif_matches(root_dir,als_subfolder=als_subfolder, tif_subfolder=masks_subfolder, contains='merged')

📂 Extracted identifier '33470-5714' from folder: als_33470-5714
✅ ALS File: data\Tschernitz\als\als_33470-5714\als_33470-5714.laz -> Matched TIF: data\Tschernitz\ground_truth_masks\forest_masks_dop_2500\als_33470-5714\merged.tif

📂 Extracted identifier '33470-5715' from folder: als_33470-5715
✅ ALS File: data\Tschernitz\als\als_33470-5715\als_33470-5715.laz -> Matched TIF: data\Tschernitz\ground_truth_masks\forest_masks_dop_2500\als_33470-5715\merged.tif

📂 Extracted identifier '33470-5716' from folder: als_33470-5716
✅ ALS File: data\Tschernitz\als\als_33470-5716\als_33470-5716.laz -> Matched TIF: data\Tschernitz\ground_truth_masks\forest_masks_dop_2500\als_33470-5716\merged.tif

📂 Extracted identifier '33471-5713' from folder: als_33471-5713
✅ ALS File: data\Tschernitz\als\als_33471-5713\als_33471-5713.laz -> Matched TIF: data\Tschernitz\ground_truth_masks\forest_masks_dop_2500\als_33471-5713\merged.tif

📂 Extracted identifier '33471-5714' from folder: als_33471-5714
✅ ALS File: data

In [6]:
root_dir = "data/Tschernitz"
folder1 = "dop"
folder2 = "ground_truth_masks/forest_masks_dop_2500"

# Match stacked TIF and ground truth maskdop
dop_matches = get_tif_file_matches(root_dir, folder1, folder2, contains2="merged")
dop_matches

Folder1 Path: data/Tschernitz\dop
Folder2 Path: data/Tschernitz\ground_truth_masks/forest_masks_dop_2500


{'data\\Tschernitz\\dop\\dop_33470-5714\\dop_33470-5714.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33470-5714\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33470-5715\\dop_33470-5715.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33470-5715\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33470-5716\\dop_33470-5716.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33470-5716\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33471-5713\\dop_33471-5713.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33471-5713\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33471-5714\\dop_33471-5714.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33471-5714\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33471-5715\\dop_33471-5715.tif': 'data\\Tschernitz\\ground_truth_masks\\forest_masks_dop_2500\\als_33471-5715\\merged.tif',
 'data\\Tschernitz\\dop\\dop_33471-5716\\dop_33471-5716.tif': 'data\\Tschernitz\\g

In [7]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np

def overlay_dop_gt(dop_path, gt_path, ax):
    """
    Overlay DOP (RGB) and Ground Truth (GT) mask for visualization.

    Args:
        dop_path (str): Path to the DOP image (RGB).
        gt_path (str): Path to the corresponding ground truth mask.
        ax (matplotlib.axes.Axes): Axis to plot the overlay.
    """
    # Load DOP (RGB image)
    with rasterio.open(dop_path) as src:
        dop = src.read([1, 2, 3])  # Select RGB bands
        dop = np.moveaxis(dop, 0, -1)  # Move channel axis to last

    # Load Ground Truth Mask
    with rasterio.open(gt_path) as src:
        gt = src.read(1)  # Read first (mask) band

    # Normalize DOP for display (assuming 8-bit images)
    dop = dop / dop.max()

    # Create overlay plot
    ax.imshow(dop)
    ax.imshow(gt, alpha=0.3)  # Red overlay for GT
    ax.set_title(f"Overlay: {os.path.basename(dop_path)}")
    ax.axis("off")


In [8]:
# **Load all DOP-GT matches**
all_samples = list(dop_matches.items())  # Convert dictionary to list of tuples



# **2) Display all samples together in a grid layout**
num_samples = len(all_samples)
cols = 5  # Set number of columns in grid
rows = (num_samples // cols) + (num_samples % cols > 0)  # Compute rows dynamically

fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))

# Flatten axes for iteration
axes = axes.flatten()

# Loop through all samples and plot
for ax, (dop_path, gt_path) in zip(axes, all_samples):
    overlay_dop_gt(dop_path, gt_path, ax)

# Hide any empty subplots
for i in range(len(all_samples), len(axes)):
    fig.delaxes(axes[i])    

plt.tight_layout()
plt.show()

: 

In [ ]:
# **1) Display each sample one by one**
for dop_path, gt_path in all_samples:
    fig, ax = plt.subplots(figsize=(5, 5))
    overlay_dop_gt(dop_path, gt_path, ax)
    plt.tight_layout()
    plt.show()

In [12]:
# Get the first element from the dictionary
first_als_gt_element = list(als_gt_matches.items())[0]

# Convert the first element to another dictionary
first_als_mask_match_dict = {first_als_gt_element[0]: first_als_gt_element[1]}

In [13]:
segmentation_map = visualize_overlays(first_als_mask_match_dict, max_images=22)

# To display the map
segmentation_map

Added segmentation mask for als_33470-5714 to the map.
Displaying map with 1 segmentation overlays.


Map(center=[51.580789499999995, 14.574251], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…